In [ ]:
import os
import warnings
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt

import condo

In [ ]:
##############################################################################
# Load image data
# ---------------


def im2mat(img):
    """Converts and image to matrix (one pixel per line)"""
    return img.reshape((img.shape[0] * img.shape[1], img.shape[2]))


def mat2im(X, shape):
    """Converts back a matrix to an image"""
    return X.reshape(shape)


def minmax(img):
    return np.clip(img, 0, 1)


# Loading images
this_file = os.path.realpath('__file__')
data_path = os.path.join(Path(this_file).parent.parent, 'data')
data_path = '../data/'
I1 = plt.imread(os.path.join(data_path, 'ocean_day.jpg')).astype(np.float32) / 256
I2 = plt.imread(os.path.join(data_path, 'ocean_sunset.jpg')).astype(np.float32) / 256
I3 = plt.imread(os.path.join(data_path, 'cape_cod.jpg')).astype(np.float32) / 256

I1seg = I1.copy().astype(str)
I1seg[0:400, :, :] = 'sky'
I1seg[400:, :, :] = 'water'
I1segshow = np.zeros((I1.shape[0], I1.shape[1]))
I1segshow[0:400, :] = 1.0

I2seg = I2.copy().astype(str)
I2seg[0:475, :, :] = 'sky'
I2seg[475:, :, :] = 'water'
I2segshow = np.zeros((I2.shape[0], I2.shape[1]))
I2segshow[0:475, :] = 1.0

I3is, I3js = np.meshgrid(range(I3.shape[0]), range(I3.shape[1]), indexing='ij')
I3segshow = np.where(I3is - 300 < I3js/5, 1, 0)
I3segshow[0:275, :] += 1
I3seg = np.where(I3is - 300 < I3js/5, "water", "sand")
I3seg[0:275, :] = "sky"

Y1 = im2mat(I1seg)[:, [0]]
Y2 = im2mat(I2seg)[:, [0]]
Y3 = im2mat(np.expand_dims(I3seg, 2))

X1 = im2mat(I1)
X2 = im2mat(I2)
X3 = im2mat(I3)

fig, axes = plt.subplots(1, 2)
axes[0].imshow(I1)
axes[1].imshow(I1segshow)
fig.savefig("figure-DayLabels.pdf", bbox_inches="tight")

fig, axes = plt.subplots(1, 2)
axes[0].imshow(I2)
axes[1].imshow(I2segshow)
fig.savefig("figure-SunsetLabels.pdf", bbox_inches="tight")


fig, axes = plt.subplots(1, 2)
axes[0].imshow(I3)
axes[1].imshow(I3segshow)
fig.savefig("figure-BeachLabels.pdf", bbox_inches="tight")

mmd_size = 20
bootstrap_fraction = 0.01
invstr = ' (Inverse Mapping)'
invstr = ''

# Feature-Shift Only (Day to Sunset)

In [ ]:
# Estimate mapping and adapt
otda12 = condo.AdapterGaussianOT(transform_type='affine')
otda12.fit(Xs=X1, Xt=X2)
xst_otda12 = otda12.transform(Xs=X1)
xts_otda12 = otda12.inverse_transform(Xt=X2)
I1t_otda12 = minmax(mat2im(xst_otda12, I1.shape))
I2t_otda12 = minmax(mat2im(xts_otda12, I2.shape))


mmder12 = condo.AdapterMMD(
    transform_type="affine",
    bootstrap_fraction=bootstrap_fraction,
    n_bootstraps=1,
    n_epochs=100,
    mmd_size=mmd_size,
    learning_rate=1e-2,
)
mmder12.fit(X1, X2)
xst_mmd12 = mmder12.transform(X1)
xts_mmd12 = mmder12.inverse_transform(X2)
I1t_mmd12 = minmax(mat2im(xst_mmd12, I1.shape))
I2t_mmd12 = minmax(mat2im(xts_mmd12, I2.shape))

cder12 = condo.ConDoAdapterKLD(
    transform_type="affine",
)
cder12.fit(X1, X2, Y1, Y2)
xst_linear12 = cder12.transform(X1)
xts_linear12 = cder12.inverse_transform(X2)
I1t_linear12 = minmax(mat2im(xst_linear12, I1.shape))
I2t_linear12 = minmax(mat2im(xts_linear12, I2.shape))

cder12 = condo.ConDoAdapterMMD(
    transform_type="affine",
    n_epochs=100,
    bootstrap_fraction=bootstrap_fraction,
    n_bootstraps=1,
    mmd_size=mmd_size,
    learning_rate=1e-2,
)
cder12.fit(X1, X2, Y1, Y2)
xst_cmmd12 = cder12.transform(X1)
xts_cmmd12 = cder12.inverse_transform(X2)
I1t_cmmd12 = minmax(mat2im(xst_cmmd12, I1.shape))
I2t_cmmd12 = minmax(mat2im(xts_cmmd12, I2.shape))

In [ ]:
# Plot transformed images
fig = plt.figure(figsize=(5, 10), constrained_layout=True)

subfigs = fig.subfigures(nrows=6, ncols=1)
subfigs[0].suptitle('Original Images');
axes = subfigs[0].subplots(nrows=1, ncols=2)
axes[0].imshow(I1)
axes[0].axis('off')
axes[0].set_title('Day')
axes[1].imshow(I2)
axes[1].axis('off')
axes[1].set_title('Sunset');

subfigs[1].suptitle('Gaussian OT');
axes = subfigs[1].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_otda12)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Sunset')
axes[1].imshow(I2t_otda12)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Sunset{invstr}')

subfigs[2].suptitle('MMD');
axes = subfigs[2].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_mmd12)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Sunset')
axes[1].imshow(I2t_mmd12)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Sunset{invstr}');

subfigs[3].suptitle('ConDo segmentation');
axes = subfigs[3].subplots(nrows=1, ncols=2)
axes[0].imshow(I1segshow)
axes[0].axis('off')
axes[0].set_title('Day pixel labels')
axes[1].imshow(I2segshow)
axes[1].axis('off')
axes[1].set_title(f'Sunset pixel labels');

subfigs[4].suptitle('ConDo Gaussian KLD');
axes = subfigs[4].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_linear12)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Sunset')
axes[1].imshow(I2t_linear12)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Sunset{invstr}');

subfigs[5].suptitle('ConDo MMD');
axes = subfigs[5].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_cmmd12)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Sunset')
axes[1].imshow(I2t_cmmd12)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Sunset{invstr}');

fig.savefig("figure-DayToSunset-inclmmd.pdf", bbox_inches="tight")

# Confounded Shift (Day to Beach)

In [ ]:
# Estimate mapping and adapt
otda13 = condo.AdapterGaussianOT(transform_type='affine')
otda13.fit(Xs=X1, Xt=X3)
xst_otda13 = otda13.transform(Xs=X1)
xts_otda13 = otda13.inverse_transform(Xt=X3)
I1t_otda13 = minmax(mat2im(xst_otda13, I1.shape))
I3t_otda13 = minmax(mat2im(xts_otda13, I3.shape))

cder13 = condo.ConDoAdapterKLD(
    transform_type="affine",
)
cder13.fit(X1, X3, Y1, Y3)
xst_linear13 = cder13.transform(X1)
xts_linear13 = cder13.inverse_transform(X3)
I1t_linear13 = minmax(mat2im(xst_linear13, I1.shape))
I3t_linear13 = minmax(mat2im(xts_linear13, I3.shape))

mmder13 = condo.AdapterMMD(
    transform_type="affine",
    bootstrap_fraction=bootstrap_fraction,
    n_epochs=100,
    learning_rate=1e-2,
    n_bootstraps=1,
    mmd_size=mmd_size,
    verbose=0,
)
mmder13.fit(X1, X3)
xst_mmd13 = mmder13.transform(X1)
xts_mmd13 = mmder13.inverse_transform(X3)
I1t_mmd13 = minmax(mat2im(xst_mmd13, I1.shape))
I3t_mmd13 = minmax(mat2im(xts_mmd13, I3.shape))

cder13 = condo.ConDoAdapterMMD(
    transform_type="affine",
    n_epochs=100,
    bootstrap_fraction=bootstrap_fraction,
    learning_rate=1e-2,
    n_bootstraps=1,
    mmd_size=mmd_size,
    verbose=0,
)
cder13.fit(X1, X3, Y1, Y3)
xst_cmmd13 = cder13.transform(X1)
xts_cmmd13 = cder13.inverse_transform(X3)
I1t_cmmd13 = minmax(mat2im(xst_cmmd13, I1.shape))
I3t_cmmd13 = minmax(mat2im(xts_cmmd13, I3.shape))

In [ ]:
# Plot transformed images
fig = plt.figure(figsize=(5, 10), constrained_layout=True)

subfigs = fig.subfigures(nrows=6, ncols=1)
subfigs[0].suptitle('Original Images');
axes = subfigs[0].subplots(nrows=1, ncols=2)
axes[0].imshow(I1)
axes[0].axis('off')
axes[0].set_title('Day')
axes[1].imshow(I3)
axes[1].axis('off')
axes[1].set_title('Beach');

subfigs[1].suptitle('Gaussian OT');
axes = subfigs[1].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_otda13)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Beach')
axes[1].imshow(I3t_otda13)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Beach{invstr}')

subfigs[2].suptitle('MMD');
axes = subfigs[2].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_mmd13)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Beach')
axes[1].imshow(I3t_mmd13)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Beach{invstr}');

subfigs[3].suptitle('ConDo segmentation');
axes = subfigs[3].subplots(nrows=1, ncols=2)
axes[0].imshow(I1segshow)
axes[0].axis('off')
axes[0].set_title('Day pixel labels')
axes[1].imshow(I3segshow)
axes[1].axis('off')
axes[1].set_title(f'Beach pixel labels');

subfigs[4].suptitle('ConDo Gaussian KLD');
axes = subfigs[4].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_linear13)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Beach')
axes[1].imshow(I3t_linear13)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Beach{invstr}');

subfigs[5].suptitle('ConDo MMD');
axes = subfigs[5].subplots(nrows=1, ncols=2)
axes[0].imshow(I1t_cmmd13)
axes[0].axis('off')
axes[0].set_title('Day $\longrightarrow$ Beach')
axes[1].imshow(I3t_cmmd13)
axes[1].axis('off')
axes[1].set_title(f'Day $\longleftarrow$ Beach{invstr}')

fig.savefig("figure-DayToBeach-inclmmd.pdf", bbox_inches="tight")

# Confounded Shift (Sunset to Beach)

In [ ]:
# Estimate mapping and adapt
otda23 = condo.AdapterGaussianOT(transform_type='affine')
otda23.fit(Xs=X2, Xt=X3)
xst_otda23 = otda23.transform(Xs=X2)
xts_otda23 = otda23.inverse_transform(Xt=X3)
I2t_otda23 = minmax(mat2im(xst_otda23, I2.shape))
I3t_otda23 = minmax(mat2im(xts_otda23, I3.shape))

mmder23 = condo.AdapterMMD(
    transform_type="affine",
    bootstrap_fraction=bootstrap_fraction,
    n_epochs=100,
    learning_rate=1e-2,
    n_bootstraps=1,
    mmd_size=mmd_size,
    verbose=0,
)
mmder23.fit(X2, X3)
xst_mmd23 = mmder23.transform(X2)
xts_mmd23 = mmder23.inverse_transform(X3)
I2t_mmd23 = minmax(mat2im(xst_mmd23, I2.shape))
I3t_mmd23 = minmax(mat2im(xts_mmd23, I3.shape))

cder23 = condo.ConDoAdapterKLD(
    transform_type="affine",
)
cder23.fit(X2, X3, Y2, Y3)
xst_linear23 = cder23.transform(X2)
xts_linear23 = cder23.inverse_transform(X3)
I2t_linear23 = minmax(mat2im(xst_linear23, I2.shape))
I3t_linear23 = minmax(mat2im(xts_linear23, I3.shape))

cder23 = condo.ConDoAdapterMMD(
    transform_type="affine",
    n_epochs=100,
    bootstrap_fraction=bootstrap_fraction,
    learning_rate=1e-2,
    n_bootstraps=1,
    mmd_size=mmd_size,
    verbose=0,
)
cder23.fit(X2, X3, Y2, Y3)
xst_cmmd23 = cder23.transform(X2)
xts_cmmd23 = cder23.inverse_transform(X3)
I2t_cmmd23 = minmax(mat2im(xst_cmmd23, I2.shape))
I3t_cmmd23 = minmax(mat2im(xts_cmmd23, I3.shape))

In [ ]:
# Plot transformed images
fig = plt.figure(figsize=(5, 10), constrained_layout=True)

subfigs = fig.subfigures(nrows=6, ncols=1)
subfigs[0].suptitle('Original Images');
axes = subfigs[0].subplots(nrows=1, ncols=2)
axes[0].imshow(I2)
axes[0].axis('off')
axes[0].set_title('Sunset')
axes[1].imshow(I3)
axes[1].axis('off')
axes[1].set_title('Beach');

subfigs[1].suptitle('Gaussian OT');
axes = subfigs[1].subplots(nrows=1, ncols=2)
axes[0].imshow(I2t_otda23)
axes[0].axis('off')
axes[0].set_title('Sunset $\longrightarrow$ Beach')
axes[1].imshow(I3t_otda23)
axes[1].axis('off')
axes[1].set_title(f'Sunset $\longleftarrow$ Beach{invstr}')

subfigs[2].suptitle('MMD');
axes = subfigs[2].subplots(nrows=1, ncols=2)
axes[0].imshow(I2t_mmd23)
axes[0].axis('off')
axes[0].set_title('Sunset $\longrightarrow$ Beach')
axes[1].imshow(I3t_mmd23)
axes[1].axis('off')
axes[1].set_title(f'Sunset $\longleftarrow$ Beach{invstr}');

subfigs[3].suptitle('ConDo segmentation');
axes = subfigs[3].subplots(nrows=1, ncols=2)
axes[0].imshow(I2segshow)
axes[0].axis('off')
axes[0].set_title('Sunset pixel labels')
axes[1].imshow(I3segshow)
axes[1].axis('off')
axes[1].set_title(f'Beach pixel labels');

subfigs[4].suptitle('ConDo Gaussian KLD');
axes = subfigs[4].subplots(nrows=1, ncols=2)
axes[0].imshow(I2t_linear23)
axes[0].axis('off')
axes[0].set_title('Sunset $\longrightarrow$ Beach')
axes[1].imshow(I3t_linear23)
axes[1].axis('off')
axes[1].set_title(f'Sunset $\longleftarrow$ Beach{invstr}');

subfigs[5].suptitle('ConDo MMD');
axes = subfigs[5].subplots(nrows=1, ncols=2)
axes[0].imshow(I2t_cmmd23)
axes[0].axis('off')
axes[0].set_title('Sunset $\longrightarrow$ Beach')
axes[1].imshow(I3t_cmmd23)
axes[1].axis('off')
axes[1].set_title(f'Sunset $\longleftarrow$ Beach{invstr}')

fig.savefig("figure-SunsetToBeach-inclmmd.pdf", bbox_inches="tight")